Quinto ejercicio de Spark Structured Streaming. Lista todos datos consumidos desde un topic de kafka cuyos datos son enviados en formato JSON como por ejemplo:
{"nombre":"JOSE ANTONIO","edad":38,"peso":75.5,"direccion":"C/ DEL LIMONERO 39, PISO 1-A"}

In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

In [ ]:
spark = SparkSession\
        .builder\
        .appName("StructuredKafkaJSON")\
        .getOrCreate()

In [ ]:
# Define schema of json
schema = StructType() \
        .add("nombre", StringType()) \
        .add("edad", IntegerType()) \
        .add("peso", FloatType()) \
        .add("direccion", StringType())

In [ ]:
# Create DataSet representing the stream of input lines from kafka
# Es necesario de antemano haber creado el topic llamado wordcount_topic
lines = spark\
        .readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "json_topic")\
        .load()\
        .selectExpr("CAST(value AS STRING)")\
        .select(from_json(col("value"), schema).alias("parsed_value"))\
        .select("parsed_value.*")

In [ ]:
# Una vez iniciado el procesamiento del flujo empezar a insertar elementos al topic de kafka
# Ver resultados en la consola donde se ejecuto el jupyter notebook
query = lines.writeStream \
        .outputMode("append") \
        .format("console") \
        .start()

query.awaitTermination()